In [1]:
import nltk
import sklearn
from platform import python_version

# Import Libraries
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from IPython.display import display_html 
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score

%matplotlib inline  

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                     for a in self.args)

    def __repr__(self):
       return '\n\n'.join(a + '\n' + repr(eval(a))
                       for a in self.args)

newline='\n'

In [2]:
#################################################
#################################################
## This Notebook uses RYAN's encoding/Scaling  ##
#################################################
#################################################

########################### Bike Dataset ##########################
from sklearn.preprocessing import OneHotEncoder 

bike_data = pd.read_csv('../input/project4set2/day.csv')
bike_data_df = pd.DataFrame(data=bike_data)

#### OHE ####
#one hot encode weathersit for sure, season probably, weekday (0-6) maybe
# label encode the date into days of the month
encoded_bike_data = bike_data_df.copy()

encoded_bike_data['dteday'] = pd.to_datetime(bike_data_df['dteday']).dt.strftime('%d')
encoded_bike_data['dteday'] = pd.to_numeric(encoded_bike_data['dteday'])

# use pd.concat to join the new columns with your original dataframe
df_season = pd.concat([encoded_bike_data,pd.get_dummies(encoded_bike_data['season'], prefix='season')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df_season.drop(['season'],axis=1, inplace=True)

df_weathersit = pd.concat([df_season, pd.get_dummies(df_season['weathersit'], prefix = 'weathersit')], axis=1)
df_weathersit.drop(['weathersit'], axis=1, inplace=True)

df_holiday = pd.concat([df_weathersit, pd.get_dummies(df_weathersit['holiday'], prefix = 'holiday')], axis=1)
df_holiday.drop(['holiday'], axis=1, inplace=True)

df_workday = pd.concat([df_holiday, pd.get_dummies(df_holiday['workingday'], prefix = 'workingday')], axis=1)
df_workday.drop(['workingday'], axis=1, inplace=True)
bike_data_proc_df = df_workday
print(bike_data_proc_df.columns)

#### Save a non-Scaled Copy ####
encodedBikeDataset=bike_data_proc_df.copy(deep=True)
unscaledEncodedBikeDataset = encodedBikeDataset.copy(deep=True)

#### Scaling ####
from sklearn.preprocessing import StandardScaler

#standardize all feature columns
#remove registered and casual?

col_names = bike_data_proc_df.columns
features = bike_data_proc_df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

#assign scaled results to the dataset copy
bike_data_proc_df[col_names] = features
#print(bike_data_proc_df)

############# Incorporate back into Jake's Notebook ##########
encodedBikeDataset=bike_data_proc_df.copy(deep=True)

bike_data_proc_df.hist(column=['temp', 'atemp','hum', 'windspeed'], figsize = (12,10), bins=50)

bikeFeatureList = list(encodedBikeDataset.columns)
bikeFeatureList.remove('cnt')
bikeFeatureList.remove('registered')
bikeFeatureList.remove('casual')

###### Used by the rest of the Notebook ######
encodedBikeDataset

Index(['instant', 'dteday', 'yr', 'mnth', 'weekday', 'temp', 'atemp', 'hum',
       'windspeed', 'casual', 'registered', 'cnt', 'season_1', 'season_2',
       'season_3', 'season_4', 'weathersit_1', 'weathersit_2', 'weathersit_3',
       'holiday_0', 'holiday_1', 'workingday_0', 'workingday_1'],
      dtype='object')


instant    dteday        yr      mnth   weekday      temp     atemp  \
0   -1.729683 -1.674108 -1.001369 -1.600161  1.498809 -0.826662 -0.679946   
1   -1.724944 -1.560522 -1.001369 -1.600161 -1.496077 -0.721095 -0.740652   
2   -1.720205 -1.446936 -1.001369 -1.600161 -0.996930 -1.634657 -1.749767   
3   -1.715466 -1.333351 -1.001369 -1.600161 -0.497782 -1.614780 -1.610270   
4   -1.710728 -1.219765 -1.001369 -1.600161  0.001366 -1.467414 -1.504971   
..        ...       ...       ...       ...       ...       ...       ...   
726  1.710728  1.279122  0.998633  1.588660  0.500513 -1.318665 -1.521108   
727  1.715466  1.392707  0.998633  1.588660  0.999661 -1.323224 -1.346690   
728  1.720205  1.506293  0.998633  1.588660  1.498809 -1.323224 -1.424344   
729  1.724944  1.619879  0.998633  1.588660 -1.496077 -1.309558 -1.490049   
730  1.729683  1.733465  0.998633  1.588660 -0.996930 -1.528225 -1.540482   

          hum  windspeed    casual  ...  season_2  season_3  season_4  \
0    1.250171  -0.387892 -0.753734  ... -0.579983 -0.588409 -0.567345   
1    0.479113   0.749602 -1.045214  ... -0.579983 -0.588409 -0.567345   
2   -1.339274   0.746632 -1.061246  ... -0.579983 -0.588409 -0.567345   
3   -0.263182  -0.389829 -1.078734  ... -0.579983 -0.588409 -0.567345   
4   -1.341494  -0.046307 -1.116627  ... -0.579983 -0.588409 -0.567345   
..        ...        ...       ...  ...       ...       ...       ...   
726  0.175807   2.061426 -0.876156  ... -0.579983 -0.588409 -0.567345   
727 -0.266238  -0.452131 -0.297567  ... -0.579983 -0.588409 -0.567345   
728  0.878392  -0.853552 -1.004407  ... -0.579983 -0.588409 -0.567345   
729 -1.015664   2.069444 -0.705640  ... -0.579983 -0.588409 -0.567345   
730 -0.354061  -0.460201 -0.596334  ... -0.579983 -0.588409 -0.567345   

     weathersit_1  weathersit_2  weathersit_3  holiday_0  holiday_1  \
0       -1.314387      1.399826     -0.171981   0.171981  -0.171981   
1       -1.314387      1.399826     -0.171981   0.171981  -0.171981   
2        0.760811     -0.714374     -0.171981   0.171981  -0.171981   
3        0.760811     -0.714374     -0.171981   0.171981  -0.171981   
4        0.760811     -0.714374     -0.171981   0.171981  -0.171981   
..            ...           ...           ...        ...        ...   
726     -1.314387      1.399826     -0.171981   0.171981  -0.171981   
727     -1.314387      1.399826     -0.171981   0.171981  -0.171981   
728     -1.314387      1.399826     -0.171981   0.171981  -0.171981   
729      0.760811     -0.714374     -0.171981   0.171981  -0.171981   
730     -1.314387      1.399826     -0.171981   0.171981  -0.171981   

     workingday_0  workingday_1  
0        1.471225     -1.471225  
1        1.471225     -1.471225  
2       -0.679706      0.679706  
3       -0.679706      0.679706  
4       -0.679706      0.679706  
..            ...           ...  
726     -0.679706      0.679706  
727     -0.679706      0.679706  
728      1.471225     -1.471225  
729      1.471225     -1.471225  
730     -0.679706      0.679706  

[731 rows x 23 columns]

In [3]:
#################################################
#################################################
## This Notebook uses RYAN's encoding/Scaling  ##
#################################################
#################################################

########################### Video Dataset ##########################
video_data = pd.read_csv('../input/project4set2/transcoding_mesurment.tsv', sep='\t')
video_data_df = pd.DataFrame(data=video_data)

#### OHE ####
video_data_noid = video_data_df.copy()
video_data_noid.drop(['id'], axis=1, inplace=True)


df_codec = pd.concat([video_data_noid, pd.get_dummies(video_data_noid['codec'], prefix = 'codec')], axis=1)
df_codec.drop(['codec'], axis=1, inplace=True)


df_o_codec = pd.concat([df_codec, pd.get_dummies(df_codec['o_codec'], prefix = 'o_codec')], axis=1)
df_o_codec.drop(['o_codec'], axis=1, inplace=True)

#add inverse here
df_o_codec['o_width'] = np.reciprocal(df_o_codec['o_width'])
df_o_codec['o_height'] = np.reciprocal(df_o_codec['o_height'])

video_data_proc_df = df_o_codec.copy()


#### Save a non Scaled Copy ###
encodedVideoDataset=video_data_proc_df.copy(deep=True)
unscaledEncodedVideoDataset = encodedVideoDataset.copy(deep=True)

#### Scaling ####
from sklearn.preprocessing import StandardScaler

video_col_names = video_data_proc_df.columns
video_features = video_data_proc_df[video_col_names]
video_scaler = StandardScaler().fit(video_features.values)
video_features = video_scaler.transform(video_features.values)

#assign scaled results to the dataset copy
video_data_proc_df[video_col_names] = video_features
#print(video_data_proc_df)

video_data_proc_df.hist(column=['duration', 'bitrate' , 'framerate' , 'i',  'p', 'b' , 'frames', 'i_size', 'p_size' , 'b_size', 'size', 'o_bitrate', 'o_framerate', 'o_height', 'o_width'
], figsize = (12,10), bins=50)

############# Incorporate back into Jake's Notebook ##########
encodedVideoDataset=video_data_proc_df.copy(deep=True)

video_data_proc_df.hist(column=['duration', 'bitrate' , 'framerate' , 'i',  'p', 'b' , 'frames', 'i_size', 'p_size' , 'b_size', 'size', 'o_bitrate', 'o_framerate', 'o_height', 'o_width'
], figsize = (12,10), bins=50)
videoFeatureList = list(encodedVideoDataset.columns)
videoFeatureList.remove('utime')
videoFeatureList.remove('umem')

###### Used by the rest of the Notebook ######
encodedVideoDataset

AttributeError: 'DataFrame' object has no attribute 'column'

In [4]:
#####################################
## Feature Selection: Bike Dataset ##
#####################################
# Target value 'cnt'
bikeMutalInfo = mutual_info_regression(encodedBikeDataset[bikeFeatureList], encodedBikeDataset['cnt'])
bikeFTest, bikePValues = f_regression(encodedBikeDataset[bikeFeatureList], encodedBikeDataset['cnt']) 

# Zip 'features' and 'values'together....
bikeMutalInfoArray = []
bikeFScoreArray = []
bikePvalueArray = []

feature_counter = 0
for feature in bikeFeatureList:
    
    # Interpretable mutual_info_regression() Array.. 
    # dependency between given feature & 'cnt'
    bikeMutalInfoArray.append((feature, bikeMutalInfo[feature_counter]))

    # Interpretable f_score Array.. 
    # including this 'feature' affects performance this much....
    bikeFScoreArray.append((feature, bikeFTest[feature_counter]))
    
    # Interpretable p_value Array.. 
    # how this feature affects target variable (by chance?)....    
    bikePvalueArray.append((feature, bikePValues[feature_counter]))
    
    feature_counter += 1
    
# Sort each array descending order
bikeMutalInfoArray.sort(key = lambda x: x[1], reverse=True)
bikeFScoreArray.sort(key = lambda x: x[1], reverse=True)
bikePvalueArray.sort(key = lambda x: x[1], reverse=True)

# Transform to DataFrame for display and label columns
bikeMutalInfoDf = pd.DataFrame(bikeMutalInfoArray,columns=['Feature', 'Mutual Info'])
bikeFScoreDf = pd.DataFrame(bikeFScoreArray,columns=['Feature', 'F-score'])
bikePValuesDf = pd.DataFrame(bikePvalueArray,columns=['Feature', 'P-Value'])

# Display all thre ordered tables
display_html(bikeMutalInfoDf)
display_html(bikeFScoreDf)
display_html(bikePValuesDf)

Feature 
 Mutual Info 
 
 
 
 
 0 
 instant 
 0.905203 
 
 
 1 
 atemp 
 0.464620 
 
 
 2 
 temp 
 0.389639 
 
 
 3 
 mnth 
 0.373819 
 
 
 4 
 yr 
 0.277507 
 
 
 5 
 season_1 
 0.195517 
 
 
 6 
 season_3 
 0.072673 
 
 
 7 
 windspeed 
 0.055760 
 
 
 8 
 hum 
 0.046359 
 
 
 9 
 weekday 
 0.045621 
 
 
 10 
 weathersit_1 
 0.043201 
 
 
 11 
 weathersit_3 
 0.043184 
 
 
 12 
 season_4 
 0.033938 
 
 
 13 
 workingday_0 
 0.023571 
 
 
 14 
 workingday_1 
 0.023571 
 
 
 15 
 weathersit_2 
 0.019176 
 
 
 16 
 holiday_0 
 0.011057 
 
 
 17 
 holiday_1 
 0.011057 
 
 
 18 
 season_2 
 0.006246 
 
 
 19 
 dteday 
 0.000000

Feature 
 F-score 
 
 
 
 
 0 
 atemp 
 482.454311 
 
 
 1 
 instant 
 476.810740 
 
 
 2 
 temp 
 473.471711 
 
 
 3 
 yr 
 344.890586 
 
 
 4 
 season_1 
 338.467598 
 
 
 5 
 season_3 
 99.459088 
 
 
 6 
 mnth 
 62.004625 
 
 
 7 
 weathersit_1 
 49.798737 
 
 
 8 
 weathersit_3 
 44.540374 
 
 
 9 
 windspeed 
 42.437842 
 
 
 10 
 weathersit_2 
 22.459057 
 
 
 11 
 season_2 
 15.921736 
 
 
 12 
 hum 
 7.461940 
 
 
 13 
 holiday_0 
 3.421441 
 
 
 14 
 holiday_1 
 3.421441 
 
 
 15 
 weekday 
 3.331091 
 
 
 16 
 season_4 
 3.149998 
 
 
 17 
 workingday_0 
 2.736742 
 
 
 18 
 workingday_1 
 2.736742 
 
 
 19 
 dteday 
 0.427336

Feature 
 P-Value 
 
 
 
 
 0 
 dteday 
 5.135051e-01 
 
 
 1 
 workingday_0 
 9.849496e-02 
 
 
 2 
 workingday_1 
 9.849496e-02 
 
 
 3 
 season_4 
 7.634430e-02 
 
 
 4 
 weekday 
 6.839081e-02 
 
 
 5 
 holiday_0 
 6.475936e-02 
 
 
 6 
 holiday_1 
 6.475936e-02 
 
 
 7 
 hum 
 6.454143e-03 
 
 
 8 
 season_2 
 7.268728e-05 
 
 
 9 
 weathersit_2 
 2.581480e-06 
 
 
 10 
 windspeed 
 1.359959e-10 
 
 
 11 
 weathersit_3 
 4.933947e-11 
 
 
 12 
 weathersit_1 
 3.971136e-12 
 
 
 13 
 mnth 
 1.243112e-14 
 
 
 14 
 season_3 
 4.795902e-22 
 
 
 15 
 season_1 
 2.225724e-62 
 
 
 16 
 yr 
 2.483540e-63 
 
 
 17 
 temp 
 2.810622e-81 
 
 
 18 
 instant 
 1.020763e-81 
 
 
 19 
 atemp 
 1.854504e-82

In [5]:
######################################
## Feature Selection: Video Dataset ##
######################################
videoMutalInfo = mutual_info_regression(encodedVideoDataset[videoFeatureList][0:20000], encodedVideoDataset[0:20000]['utime'],n_neighbors=3)
videoFTest, videoPValues = f_regression(encodedVideoDataset[videoFeatureList], encodedVideoDataset['utime']) 

# Zip 'features' and 'values'together...
videoMutalInfoArray = []
videoFScoreArray = []
videoPvalueArray = []

feature_counter = 0
for feature in videoFeatureList:
    
    # Interpretable mutual_info_regression() Array.. 
    # dependency between given feature & 'utime'
    videoMutalInfoArray.append((feature, videoMutalInfo[feature_counter]))

    # Interpretable f_score Array.. 
    # including this 'feature' affects performance this much....
    videoFScoreArray.append((feature, videoFTest[feature_counter]))
    
    # Interpretable p_value Array.. 
    # how this feature affects target variable (by chance?)....    
    videoPvalueArray.append((feature, videoPValues[feature_counter]))
    
    feature_counter += 1
    
# Sort each array descending order
videoMutalInfoArray.sort(key = lambda x: x[1], reverse=True)
videoFScoreArray.sort(key = lambda x: x[1], reverse=True)
videoPvalueArray.sort(key = lambda x: x[1], reverse=True)

# Transform to DataFrame for display and label columns
videoMutalInfoDf = pd.DataFrame(videoMutalInfoArray,columns=['Feature', 'Mutual Info'])
videoFScoreDf = pd.DataFrame(videoFScoreArray,columns=['Feature', 'F-score'])
videoPValuesDf = pd.DataFrame(videoPvalueArray,columns=['Feature', 'P-Value'])

# Display all thre ordered tables
display_html(videoMutalInfoDf)
display_html(videoFScoreDf)
display_html(videoPValuesDf)

NameError: name 'encodedVideoDataset' is not defined

In [ ]:
#############
## Methods ##
#############

def Calcualte_Optimal_Degree(model, X, Y, degreeList):
    
    RMSE_Degree_List = []
    LOOP_COUNTER = 1
    
    for degree in degreeList:
        
        
        # Add the polynomial features
        print(f'Using {degree} Polynomial degrees for this run... {newline}')
        print(f'Feature Matrix ORIGINAL Shape (datapoints, features): {X.shape}')
        polynomial_features = PolynomialFeatures(degree=degree)
        X_poly = polynomial_features.fit_transform(X)
        print(f'Feature Matrix POLYNOMIAL Shape (datapoints, features): {X_poly.shape}{newline}')

        # Perform 10-Fold CV: Returns list of negative RMSE for each run (10)
        # If model is Lasso() or Ridge(), find the optimal penalty value!
        if isinstance(model, LinearRegression):
            cv10_neg_RMSE = cross_val_score(model, X_poly, Y, scoring='neg_root_mean_squared_error', cv=10)
        
        else:
            # Perform a grid search to get optimal 'alpha' value
            params = {'alpha':[1e-5,1e-4,1e-3,1e-2,0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5, 10,20,30,40,50,100, 200, 300, 400,500]}
            gridSearch = GridSearchCV(model ,params, scoring='neg_root_mean_squared_error', cv=10)
            gridSearch.fit(X_poly, Y)
            
            print(f'Grid Search Results... Best Penalty Value (alpha): {gridSearch.best_params_} ')
            print(f'Grid Search Results... Best Penalty Value (alpha): {gridSearch.best_score_*-1} {newline}')
            
            # Perform Cross Validation with the optimal penalty value
            optimalAlpha = gridSearch.best_params_['alpha']
            model.alpha= optimalAlpha
            cv10_neg_RMSE = cross_val_score(model, X_poly, Y, scoring='neg_root_mean_squared_error', cv=10)            
            
        # Calculate average RMSE for this 10-Fold run. Muliplty by (-1) to make positive
        avgRMSE = np.mean(cv10_neg_RMSE) *-1
        print(f'Average RMSE for this KFold run {LOOP_COUNTER}/{len(degreeList)}: {avgRMSE}{newline}')
    
        # Append to Running List
        RMSE_Degree_List.append(avgRMSE)
        
        # Update Counter
        LOOP_COUNTER +=1
        print('='*40)
        
    return RMSE_Degree_List

In [ ]:
########################################################################################
## Optimal 'degree' : Oridnal Linear Regression, Standardized Features,  All Features ##
########################################################################################

degreeList = [1,2,3]
model = LinearRegression(normalize=False)
X = encodedBikeDataset[bikeFeatureList]
Y = encodedBikeDataset['cnt']

RMSEList = Calcualte_Optimal_Degree(model, X, Y, degreeList)

print(f'Average RMSE vs. Polynomial Degrees: {degreeList}')
RMSEFrame = pd.DataFrame(RMSEList, columns=['Average RMSE'], index=degreeList)
display_html(RMSEFrame)

plt.plot(degreeList,RMSEList)
plt.title('Average RMSE vs. Polynomial Degree')
plt.xlabel('Polynomial Degrees')
plt.ylabel('Average RMSE')

In [ ]:
########################################################################################
## Optimal 'degree' : Oridnal Linear Regression, Standardized Features,  All Features ##
########################################################################################

degreeList = [1,2,3]
model = Lasso(normalize=False, max_iter=1000)
X = encodedBikeDataset[bikeFeatureList]
Y = encodedBikeDataset['cnt']

RMSEList = Calcualte_Optimal_Degree(model, X, Y, degreeList)

print(f'Average RMSE vs. Polynomial Degrees: {degreeList}')
RMSEFrame = pd.DataFrame(RMSEList, columns=['Average RMSE'], index=degreeList)
display_html(RMSEFrame)

plt.plot(degreeList,RMSEList)
plt.title('Average RMSE vs. Polynomial Degree')
plt.xlabel('Polynomial Degrees')
plt.ylabel('Average RMSE')

In [ ]:
########################################################################################
## Optimal 'degree' : Oridnal Linear Regression, Standardized Features,  All Features ##
########################################################################################

degreeList = [1,2,3]
model = Ridge(normalize=False, max_iter=1000)
X = encodedBikeDataset[bikeFeatureList]
Y = encodedBikeDataset['cnt']

RMSEList = Calcualte_Optimal_Degree(model, X, Y, degreeList)

print(f'Average RMSE vs. Polynomial Degrees: {degreeList}')
RMSEFrame = pd.DataFrame(RMSEList, columns=['Average RMSE'], index=degreeList)
display_html(RMSEFrame)

plt.plot(degreeList,RMSEList)
plt.title('Average RMSE vs. Polynomial Degree')
plt.xlabel('Polynomial Degrees')
plt.ylabel('Average RMSE')

In [ ]:
########################################################################################
## Optimal 'degree' : Oridnal Linear Regression, Standardized Features,  All Features ##
########################################################################################

degreeList = [1,2,3]
model = LinearRegression(normalize=False)
X = encodedVideoDataset[videoFeatureList]
Y = encodedVideoDataset['utime']

RMSEList = Calcualte_Optimal_Degree(model, X, Y, degreeList)

print(f'Average RMSE vs. Polynomial Degrees: {degreeList}')
RMSEFrame = pd.DataFrame(RMSEList, columns=['Average RMSE'], index=degreeList)
display_html(RMSEFrame)

plt.plot(degreeList,RMSEList)
plt.title('Average RMSE vs. Polynomial Degree')
plt.xlabel('Polynomial Degrees')
plt.ylabel('Average RMSE')

In [ ]:
########################################################################################
## Optimal 'degree' : Oridnal Linear Regression, Standardized Features,  All Features ##
########################################################################################

degreeList = [1,2,3]
model = Lasso(normalize=False, max_iter=1)
X = encodedVideoDataset[videoFeatureList]
Y = encodedVideoDataset['utime']

RMSEList = Calcualte_Optimal_Degree(model, X, Y, degreeList)

print(f'Average RMSE vs. Polynomial Degrees: {degreeList}')
RMSEFrame = pd.DataFrame(RMSEList, columns=['Average RMSE'], index=degreeList)
display_html(RMSEFrame)

plt.plot(degreeList,RMSEList)
plt.title('Average RMSE vs. Polynomial Degree')
plt.xlabel('Polynomial Degrees')
plt.ylabel('Average RMSE')

In [ ]:
########################################################################################
## Optimal 'degree' : Oridnal Linear Regression, Standardized Features,  All Features ##
########################################################################################

degreeList = [1,2,3]
model = Ridge(normalize=False, max_iter=1000)
X = encodedVideoDataset[videoFeatureList]
Y = encodedVideoDataset['utime']

RMSEList = Calcualte_Optimal_Degree(model, X, Y, degreeList)

print(f'Average RMSE vs. Polynomial Degrees: {degreeList}')
RMSEFrame = pd.DataFrame(RMSEList, columns=['Average RMSE'], index=degreeList)
display_html(RMSEFrame)

plt.plot(degreeList,RMSEList)
plt.title('Average RMSE vs. Polynomial Degree')
plt.xlabel('Polynomial Degrees')
plt.ylabel('Average RMSE')

In [ ]:
#comment